In [1]:
import platform 
from pathlib import Path

import numpy as np
import torch
import torchvision
import coremltools as ct
from PIL import Image

from alphapose.utils.config import update_config
from alphapose.models import builder

In [2]:
cfg_path = "./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml"
cfg = update_config(cfg_path)

model_path = "./exp/good_2601-256x192_res50_lr1e-3_1x.yaml/model_26.pth"
torch_model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
torch_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [3]:
torch_model.eval()

FastPose(
  (preact): SEResnet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)

In [4]:
# Trace with random data
random_input = torch.rand(1, 3, 256, 192)
traced_model = torch.jit.trace(torch_model, random_input)

In [5]:
# pytorch 1.6.0
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(name="image", shape=(1, 3, 256, 192))],
)

# Set feature descriptions (these show up as comments in XCode)
model.input_description["image"] = "Person image to detecto keypoints and its radius"

# Set model author name
model.author = 'Konstantin Slavnov'

# Set a short description for the Xcode UI
model.short_description = "Predicts person's keypoints and its size (aka radius)."

# Set a version for the model
model.version = "0.1"

Translating MIL ==> MLModel Ops: 100%|██████████| 782/782 [00:03<00:00, 240.37 ops/s] 


In [6]:
spec = model.get_spec()
assert spec.description.output[0].name == "797", spec.description.output[0]
assert spec.description.output[1].name == "816", spec.description.output[1]
ct.utils.rename_feature(spec, "797", "joints")
ct.utils.rename_feature(spec, "816", "radius")
mlmodel_modif = ct.models.MLModel(spec)

In [7]:
mlmodel_modif

input {
  name: "image"
  shortDescription: "Person image to detecto keypoints and its radius"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 256
      shape: 192
      dataType: FLOAT32
    }
  }
}
output {
  name: "joints"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "radius"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  shortDescription: "Predicts person\'s keypoints and its size (aka radius)."
  versionString: "0.1"
  author: "Konstantin Slavnov"
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.6.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.0"
  }
}

In [8]:
# Save model
export_dir = Path("export")
export_dir.mkdir(exist_ok=True)
model_path = export_dir / "fast_res50_256x192.mlmodel"
mlmodel_modif.save(str(model_path))

In [10]:
def preprocess(img):
    img = np.array(img)
    img = img.astype(np.float) / 255
    img[0] -= 0.406
    img[1] -= 0.457
    img[2] -= 0.480
    img = img.transpose([2, 0, 1])[None, :, :, :]
    return img

img = preprocess(Image.open("test_img.png"))
img = torch.Tensor(img)
torch_out = torch_model(img)

In [12]:
import pickle
with open(str(model_path) + "_torch_out.pkl", "wb") as f:
    pickle.dump(torch_out, f)

In [ ]:
# failed to work 
if platform.system() != 'Darwin':
    import tvm
    from tvm import te
    import tvm.relay as relay
    from tvm.contrib.download import download_testdata
    img = Image.open("test_img.png")
    target = "llvm"
    
    img = np.array(img)
    x = np.transpose(img, (2, 0, 1))[np.newaxis, :]
    shape_dict = {"": x.shape}
    
    # Parse CoreML model and convert into Relay computation graph
    mod, params = relay.frontend.from_coreml(mlmodel_modif, shape_dict)

    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target, params=params)